In [15]:
import cv2
import skimage.io as io
import numpy as np
from skimage.morphology import binary_erosion, binary_dilation, binary_closing,skeletonize, thin
from scipy.signal import convolve2d
from statistics import mode,variance
from math import sqrt
from skimage.measure import find_contours
import matplotlib.pyplot as plt 
from skimage import data, color, img_as_ubyte 
from skimage.feature import canny 
from skimage.transform import hough_ellipse 
from skimage.draw import ellipse_perimeter 
from skimage.draw import rectangle
from skimage.morphology import disk
from scipy.spatial.distance import euclidean


In [16]:
def showBinaryImg(img):
    cv2.imshow("Images",img*255)
    cv2.waitKey(0) 
    cv2.destroyAllWindows()
def showImg(img):
    cv2.imshow("Images",img)
    cv2.waitKey(0) 
    cv2.destroyAllWindows()

In [17]:
def opening(img,ST):
    imgEroded=cv2.erode(img,ST)
    return cv2.dilate(imgEroded,ST)
def closing(img,ST):
    imgDilated=cv2.dilate(img,ST)
    return cv2.erode(imgDilated,ST)

In [18]:
import numpy as np

from skimage.util.shape import view_as_windows

def feng_threshold(img, w_size1=15, w_size2=30,
                   k1=0.15, k2=0.01, alpha1=0.1):
    """ Runs the Feng's thresholding algorithm.
    Reference:
    Algorithm proposed in: Meng-Ling Feng and Yap-Peng Tan, “Contrast adaptive
    thresholding of low quality document images”, IEICE Electron. Express,
    Vol. 1, No. 16, pp.501-506, (2004).
    Modifications: Using integral images to compute the local mean and the
    standard deviation
    @param img: The input image. Must be a gray scale image
    @type img: ndarray
    @param w_size1: The size of the primary local window to compute
        each pixel threshold. Should be an odd window
    @type w_size1: int
    @param w_size2: The size of the secondary local window to compute
        the dynamic range standard deviation. Should be an odd window
    @type w_size2: int
    @param k1: Parameter value that lies in the interval [0.15, 0.25].
    @type k1: float
    @param k2: Parameter value that lies in the interval [0.01, 0.05].
    @type k2: float
    @param alpha1: Parameter value that lies in the interval [0.15, 0.25].
    @type alpha1: float
    @return: The estimated local threshold for each pixel
    @rtype: ndarray
    """
    # Obtaining rows and cols
    rows, cols = img.shape
    i_rows, i_cols = rows + 1, cols + 1

    # Computing integral images
    # Leaving first row and column in zero for convenience
    integ = np.zeros((i_rows, i_cols), np.float)
    sqr_integral = np.zeros((i_rows, i_cols), np.float)

    integ[1:, 1:] = np.cumsum(np.cumsum(img.astype(np.float), axis=0), axis=1)
    sqr_img = np.square(img.astype(np.float))
    sqr_integral[1:, 1:] = np.cumsum(np.cumsum(sqr_img, axis=0), axis=1)

    # Defining grid
    x, y = np.meshgrid(np.arange(1, i_cols), np.arange(1, i_rows))

    # Obtaining local coordinates
    hw_size = w_size1 // 2
    x1 = (x - hw_size).clip(1, cols)
    x2 = (x + hw_size).clip(1, cols)
    y1 = (y - hw_size).clip(1, rows)
    y2 = (y + hw_size).clip(1, rows)

    # Obtaining local areas size
    l_size = (y2 - y1 + 1) * (x2 - x1 + 1)

    # Computing sums
    sums = (integ[y2, x2] - integ[y2, x1 - 1] -
            integ[y1 - 1, x2] + integ[y1 - 1, x1 - 1])
    sqr_sums = (sqr_integral[y2, x2] - sqr_integral[y2, x1 - 1] -
                sqr_integral[y1 - 1, x2] + sqr_integral[y1 - 1, x1 - 1])

    # Computing local means
    means = sums / l_size

    # Computing local standard deviation
    stds = np.sqrt(sqr_sums / l_size - np.square(means))

    # Obtaining windows
    padded_img = np.ones((rows + w_size1 - 1, cols + w_size1 - 1)) * np.nan
    padded_img[hw_size: -hw_size, hw_size: -hw_size] = img

    winds = view_as_windows(padded_img, (w_size1, w_size1))

    # Obtaining maximums and minimums
    mins = np.nanmin(winds, axis=(2, 3))

    # Obtaining local coordinates for std range calculations
    hw_size = w_size2 // 2
    x1 = (x - hw_size).clip(1, cols)
    x2 = (x + hw_size).clip(1, cols)
    y1 = (y - hw_size).clip(1, rows)
    y2 = (y + hw_size).clip(1, rows)

    # Obtaining local areas size
    l_size = (y2 - y1 + 2) * (x2 - x1 + 2)

    # Computing sums
    sums = (integ[y2, x2] - integ[y2, x1 - 1] -
            integ[y1 - 1, x2] + integ[y1 - 1, x1 - 1])
    sqr_sums = (sqr_integral[y2, x2] - sqr_integral[y2, x1 - 1] -
                sqr_integral[y1 - 1, x2] + sqr_integral[y1 - 1, x1 - 1])

    # Computing local means2
    means2 = sums / l_size
    # Computing standard deviation range
    std_ranges = np.sqrt(sqr_sums / l_size - np.square(means2))

    # Computing normalized standard deviations and extra alpha parameters
    n_stds = stds / std_ranges
    n_sqr_std = np.square(n_stds)
    alpha2 = k1 * n_sqr_std
    alpha3 = k2 * n_sqr_std

    thresholds = ((1 - alpha1) * means + alpha2 * n_stds
                  * (means - mins) + alpha3 * mins)
    for x in range(img.shape[0]):
        for y in range(img.shape[1]):
            if(img[x][y]<thresholds[x][y]):
                img[x][y]=0
            else:
                img[x][y]=1
    return img


In [19]:
img= cv2.imread("aloha.png",0) 
if(np.amax(img)!=1):
    img=(img/255)
img=feng_threshold(img)
img2= np.copy(img)
img=1-img
proj = np.sum(img,1)
maxProjection = np.max(proj) 
result = np.zeros(img.shape) 
width=img.shape[1] 
# Draw a line for each row 
for row in range(img.shape[0]):
    result[row,0:int(proj[row])]=1    

cv2.imwrite('result.png', result) 
result=1-result

<ipython-input-18-ca460b631c7f>:67: RuntimeWarning: invalid value encountered in sqrt
  stds = np.sqrt(sqr_sums / l_size - np.square(means))


In [21]:
def filterPeaks(peaksMids,widths):
    newPeaks=[]
    newWidth=[]
    sumW=0
    for i in widths:
        sumW+=i
    avgW=sumW/len(widths)
    #if i dont have flase peaks for clusters then this will result in a problem
    var = variance(widths)
    print("var:",var)
    for i in range(len(peaksMids)):
        if(widths[i]>=avgW or (var<1000 and (avgW-widths[i])<20)):
            newPeaks.append(peaksMids[i])
            newWidth.append(widths[i])
    return newWidth,newPeaks
def imgStaffSegments(img,peaksMids,widths,staffH):
    segments=[]
    staffH=int(staffH)
    #TODO:add double the staff space instead of 5
    for i in range(len(peaksMids)):
        segments.append(img2[peaksMids[i]-int(widths[i]/2)-2*staffH:peaksMids[i]+int(widths[i]/2)+2*staffH,:])
    return segments

In [22]:
def calcStaffPos(rowHist,isSigment):
    n,m=rowHist.shape
    if(isSigment):
        thres=20
    else:
        thres=int(maxProjection*0.6)
    start=end=0
    width=[]
    peaksMid=[]
    for i in range(n-1):
        if(rowHist[i][thres]!=rowHist[i+1][thres]):
            if rowHist[i][thres]==1:
                #end of zeros
                start=i
            else:
                #end of ones
                end=i+1
                peaksMid.append(int((start+end)/2))
                width.append((end-start))
    
    return np.array(width),np.array(peaksMid)


width,peaksMids=calcStaffPos(result,False)
maxSpace=0
for i in range(len(peaksMids)):
    if(i%5<4 and i+1<len(peaksMids)and peaksMids[i+1]-peaksMids[i]>maxSpace):
        maxSpace=peaksMids[i+1]-peaksMids[i]
        maxSpace=maxSpace-(width[i]/2+width[i+1]/2)

widthSegments,peakSegments=calcStaffPos(result,True)
widthSegments,peakSegments=filterPeaks(peakSegments,widthSegments)
imgStaffSegments=imgStaffSegments(img2,peakSegments,widthSegments,maxSpace)

imgindex=0
for i in imgStaffSegments:
    cv2.imwrite("segments/"+str(imgindex)+".png",i*255)
    imgindex+=1



var: 1124


In [23]:
def removeStaffRow(imgOriginal,midPoint,curWidth):
    thresPixel=curWidth
    for i in range(imgOriginal.shape[1]):
        pixelSum= sum(imgOriginal[midPoint-curWidth:midPoint+curWidth,i:i+1])
        if(pixelSum<=thresPixel):
           # print(imgOriginal[midPoint-curWidth:midPoint+curWidth,i:i+1])
            imgOriginal[midPoint-curWidth:midPoint+curWidth,i:i+1]=0
    return imgOriginal

img2=1-img2
ST=np.ones((1,3))
img2=cv2.dilate(img2,ST)

for i in range(len(peaksMids)):
    img2=removeStaffRow(img2,peaksMids[i],width[i])

ST=np.ones((2,1))
img2=cv2.dilate(img2,ST)
img2=1-img2
cv2.imwrite("outing.png",img2*255)


True

In [24]:
contours = find_contours(img2, 0.8)
imgContours=[]
index=0
for contour in contours:
    x = contour[:,1]
    y = contour[:,0]
    [Xmin, Xmax, Ymin, Ymax] = [np.amin(x), np.amax(x), np.amin(y), np.amax(y)]
    rr, cc = rectangle(start = (Ymin,Xmin), end = (Ymax,Xmax), shape=img.shape)
    imgSymbol=img2[int(Ymin):int(Ymax+1),int(Xmin):int(Xmax+1)]
    imgContours.append([Ymin,Xmin])
    cv2.imwrite("contours/"+str(index)+".png",imgSymbol*255)
    index+=1
imgContours=np.array(imgContours)

In [25]:
segment = peaksMids[0:5] 
widths = width[0:5]
spaces=[]
for i in range(segment.shape[0]):
    if(i==0):
        continue
    spaces.append(int(((segment[i]-segment[i-1])-int(widths[i]/2+widths[i-1]/2))))
spaces=np.array(spaces)  
def getNoteName(row,col):
    note=''
    threshold=1
    if(row==segment[0] or row==segment[0]+threshold or row==segment[0]-threshold):
        note="f2"
    if(row==segment[1] or row==segment[1]+threshold or row==segment[1]-threshold):
        note="d2"
    if(row==segment[2] or row==segment[2]+threshold or row==segment[2]-threshold):
        note="b"
    if(row==segment[3] or row==segment[3]+threshold or row==segment[3]-threshold):
        note="g"
    if(row==segment[4] or row==segment[4]+threshold or row==segment[4]-threshold):
        note="e"
    if(note==''):
        if(row>segment[0] and row<segment[1]):
            note="e2"
        if(row>segment[1] and row<segment[2]):
            note="c2"
        if(row>segment[2] and row<segment[3]):
            note="a"
        if(row>segment[3] and row<segment[4]):
            note="f"
    if(note==''):
        line6=segment[4]+spaces[3]+widths[4]
        line0=segment[0]-spaces[0]-widths[0]
        linemin1=line0-spaces[0]-widths[0]
        if(row==line6 or row==line6+threshold or row==line6-threshold):
            note='c'
        if(row==line0 or row==line0+threshold or row==line0-threshold):
            note='a2'
        if(note==''):
            if(row<segment[0] and row>line0):
                note="g2"
            if(row<line0 and row>linemin1):
                note="b2"
            if(row>segment[4] and row<line6):
                note='d'
    return note
def detectBlackNoteHeadStick(symbol,index,contour):
    headOrigin=[0,int(symbol.shape[1]/2)]
    symbol=1-symbol
    proj = np.sum(symbol,1)
    for row in range(int(symbol.shape[0])):
        if(proj[row]<int(symbol.shape[1]/2)):
            symbol[row]=0
    proj = np.sum(symbol,1)
    start=0
    getIn=False
    end=0
    for row in range(int(symbol.shape[0])):
        if(proj[row]>0 and start==0):
            start=row
            getIn=True
        if(getIn and proj[row]==0):
            end=row
        if(row==symbol.shape[0]-1 and end==0):
            end=row
    headOrigin[0]=int((start+end)/2)
    headOrigin[0]=int(headOrigin[0]+contour[0])
    headOrigin[1]=int(headOrigin[1]+contour[1])
    getNoteName(headOrigin[0],headOrigin[1])
    



In [916]:
index=2
symbol=cv2.imread("imgs/"+str(index)+".png",0)
if(np.amax(symbol)!=1):
    symbol=(symbol/255)
symbol=feng_threshold(symbol)
detectBlackNoteHeadStick(symbol,index,imgContours[index])